In [12]:
# timelines_daily_pm25_and_ph_aqi_weekpart_bottomlegend.py

# --- Imports
import os, glob, re
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

# --- Paths
DATA_DIR = r"C:\Users\HP\Documents\SpatialCARE\Daily\DailyGPKG"
OUT_DIR  = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline"
os.makedirs(OUT_DIR, exist_ok=True)

# --- Column helpers
PM_CANDS        = ["pm25","PM25","PM_25","PM2_5","PM2.5"]
STATION_CANDS   = ["stations","station","Station","STATION"]
LON_CANDS       = ["longitude","lon","LONG","x"]
LAT_CANDS       = ["latitude","lat","LAT","y"]

def pick(cols, cands):
    for c in cands:
        if c in cols: return c
    return None

def safe_filename(s):
    s = str(s)
    s = re.sub(r"[^\w\s\-]+", "", s)
    return s.strip().replace(" ", "_")

# ==============================
# WHO (2021) PM2.5 — 24-hour guideline only
# ==============================
WHO_PM25_24H = 15.0  # µg/m³

def add_who_24h(ax):
    ax.axhline(WHO_PM25_24H, linestyle="-", linewidth=1.6, color="#2b8cbe", alpha=0.95)

# ==============================
# Philippine AQI for PM2.5 (24-hr) — DAO 2020-14
# ==============================
PH_AQI_LABELS = [
    "Good (0–50)",
    "Fair (51–100)",
    "Unhealthy for sensitive groups (101–150)",
    "Very unhealthy (151–200)",
    "Acutely unhealthy (201–300)",
    "Emergency (301–500)",
]
PH_AQI_COLORS = ["#00E400","#FFFF00","#FF7E00","#FF0000","#8F3F97","#7E0023"]
NO_DATA_LABEL = "No data"
NO_DATA_COLOR = "#bdbdbd"

_PH_AQI_BANDS = [(0,50),(51,100),(101,150),(151,200),(201,300),(301,500)]
_PH_PM25_BREAKS = [(0.0,25.0),(25.1,35.0),(35.1,45.0),(45.1,55.0),(55.1,90.0),(91.0, float("inf"))]

def _trunc01(x):  # truncate to 0.1 µg/m³ before converting
    return (float(x)*10)//1 / 10.0

def pm25_to_ph_aqi(x):
    v = pd.to_numeric(x, errors="coerce")
    if v is None or not np.isfinite(v): return np.nan
    v = max(0.0, _trunc01(v))
    for (Cl, Ch), (Il, Ih) in zip(_PH_PM25_BREAKS, _PH_AQI_BANDS):
        if (Cl <= v <= Ch) or (np.isinf(Ch) and v >= Cl):
            if np.isfinite(Ch):
                return (Ih-Il)/(Ch-Cl) * (v-Cl) + Il
            # open-ended emergency band; interpolate to cap
            ceiling = 150.0
            vv = min(v, ceiling)
            aqi = (500-301)/(ceiling-91.0) * (vv-91.0) + 301
            return min(500.0, aqi)
    return np.nan

def aqi_to_ph_cat_color(aqi):
    if not np.isfinite(aqi): return NO_DATA_LABEL, NO_DATA_COLOR
    a = float(aqi)
    if a <= 50:   return PH_AQI_LABELS[0], PH_AQI_COLORS[0]
    if a <= 100:  return PH_AQI_LABELS[1], PH_AQI_COLORS[1]
    if a <= 150:  return PH_AQI_LABELS[2], PH_AQI_COLORS[2]
    if a <= 200:  return PH_AQI_LABELS[3], PH_AQI_COLORS[3]
    if a <= 300:  return PH_AQI_LABELS[4], PH_AQI_COLORS[4]
    return PH_AQI_LABELS[5], PH_AQI_COLORS[5]

# ==============================
# Load daily GPKGs and build tidy table (date, station, pm25)
# ==============================
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

rows_daily = []
for f in files:
    day = os.path.splitext(os.path.basename(f))[0].replace("date_","")
    g = gpd.read_file(f)

    pm_col = pick(g.columns, PM_CANDS)
    if pm_col is None:
        print("Skip (no PM2.5):", f); continue

    # Station name; fallbacks if absent
    st_col = pick(g.columns, STATION_CANDS)
    if st_col is None:
        lon_col = pick(g.columns, LON_CANDS)
        lat_col = pick(g.columns, LAT_CANDS)
        if lon_col and lat_col:
            st_series = (g[lat_col].round(5).astype(str) + "," + g[lon_col].round(5).astype(str))
        elif "geometry" in g.columns and not g.geometry.is_empty.all():
            st_series = (g.geometry.y.round(5).astype(str) + "," + g.geometry.x.round(5).astype(str))
        else:
            st_series = g.index.astype(str)
    else:
        st_series = g[st_col].astype(str)

    pm = pd.to_numeric(g[pm_col], errors="coerce").clip(lower=0)

    for s, v in zip(st_series, pm):
        if pd.notna(v):
            rows_daily.append({
                "date": pd.to_datetime(day, errors="coerce"),
                "station": s,
                "pm25": float(v)
            })

df = pd.DataFrame(rows_daily).dropna(subset=["date","pm25"])
if df.empty:
    raise SystemExit("No usable station PM data after cleaning.")

# Average duplicates (same station & day)
df = (df.groupby(["station","date"], as_index=False)["pm25"]
        .mean()
        .sort_values(["station","date"]))

# Weekend / Weekday flag (based on daily date)
df["is_weekend"] = df["date"].dt.weekday >= 5
df["weekpart"]   = np.where(df["is_weekend"], "Weekend", "Weekday")

# Compute PH AQI & category/color
df["aqi_ph"] = df["pm25"].apply(pm25_to_ph_aqi).round(0)
cat_col = df["aqi_ph"].apply(aqi_to_ph_cat_color)
df["aqi_label"] = [c[0] for c in cat_col]
df["aqi_color"] = [c[1] for c in cat_col]

# Save audit CSV
csv_out = os.path.join(OUT_DIR, "daily_PM25_and_PHAQI_per_station_2025.csv")
df[["station","date","pm25","weekpart","aqi_ph","aqi_label"]].to_csv(csv_out, index=False)
print("Saved CSV:", csv_out)

# --- Global y-limits (so panels are comparable across stations)
pm_global_max = float(df["pm25"].max())
pm_ymax = np.ceil(pm_global_max / 5.0) * 5.0  # nearest 5 up

# --- Colors/markers for weekpart
WEEKPART_STYLE = {
    "Weekday": dict(color="#1f77b4", marker="o"),  # blue circle
    "Weekend": dict(color="#d62728", marker="D"),  # red diamond
}

# ==============================
# Plot per station: two subplots (PM2.5 vs PH-AQI) + bottom legend (outside)
# ==============================
for station, d in df.groupby("station", sort=False):
    d = d.sort_values("date")

    fig, axes = plt.subplots(1, 2, figsize=(12, 4.6), dpi=150, sharex=True)
    ax_pm, ax_aqi = axes

    # --- Left: PURE PM2.5
    for wp, dd in d.groupby("weekpart", sort=False):
        style = WEEKPART_STYLE.get(wp, dict(color="#555555", marker="o"))
        ax_pm.plot(dd["date"], dd["pm25"], linewidth=1.6, color=style["color"], alpha=0.85, zorder=1)
        ax_pm.scatter(dd["date"], dd["pm25"], s=26,
                      facecolors=style["color"], edgecolors="white", linewidths=0.5,
                      marker=style["marker"], zorder=2, label=wp)
    ax_pm.set_title(f"PM₂.₅ (µg/m³) — {station}")
    ax_pm.set_ylabel("µg/m³"); ax_pm.set_xlabel("Date")
    ax_pm.set_ylim(0, max(5, pm_ymax))
    ax_pm.grid(alpha=0.3, linestyle="--", linewidth=0.6)
    add_who_24h(ax_pm)

    # --- Right: PH-AQI
    ax_aqi.plot(d["date"], d["aqi_ph"], color="#666666", linewidth=1.1, alpha=0.7, zorder=1)
    for wp, dd in d.groupby("weekpart", sort=False):
        marker = WEEKPART_STYLE.get(wp, dict(marker="o"))["marker"]
        ax_aqi.scatter(dd["date"], dd["aqi_ph"],
                       c=dd["aqi_color"], s=28,
                       edgecolors="black", linewidths=0.4,
                       marker=marker, zorder=2, label=wp)
    ax_aqi.set_title("PH AQI (PM₂.₅, 24-hr)")
    ax_aqi.set_ylabel("AQI"); ax_aqi.set_xlabel("Date")
    ax_aqi.set_ylim(0, 500)
    ax_aqi.grid(alpha=0.3, linestyle="--", linewidth=0.6)

    # --- Build a single combined legend placed BELOW the figure
    # Weekpart markers
    weekpart_handles = [
        Line2D([0],[0], color=WEEKPART_STYLE["Weekday"]["color"], marker=WEEKPART_STYLE["Weekday"]["marker"],
               lw=1.6, label="Weekday"),
        Line2D([0],[0], color=WEEKPART_STYLE["Weekend"]["color"], marker=WEEKPART_STYLE["Weekend"]["marker"],
               lw=1.6, label="Weekend"),
    ]
    # WHO line
    who_handle = Line2D([0],[0], color="#2b8cbe", lw=1.6, label="WHO 24-hr: 15 µg/m³")
    # AQI category patches
    aqi_patches = [mpatches.Patch(color=c, label=l) for l, c in zip(PH_AQI_LABELS, PH_AQI_COLORS)]

    all_handles = weekpart_handles + [who_handle] + aqi_patches

    # Reserve bottom space and place legend outside
    plt.subplots_adjust(bottom=0.32, wspace=0.25)
    fig.legend(handles=all_handles,
               loc="upper center",
               bbox_to_anchor=(0.5, 0.20),  # push below the axes area
               ncol=3, frameon=True, fontsize=8, title="Legend")

    out_png = os.path.join(OUT_DIR, f"daily_PM25_and_PHAQI_weekpart_{safe_filename(station)}.png")
    fig.savefig(out_png, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", out_png)

Saved CSV: C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline\daily_PM25_and_PHAQI_per_station_2025.csv
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline\daily_PM25_and_PHAQI_weekpart_Brgy_San_Antonio_Fire_and_Rescue_Pasig_City.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline\daily_PM25_and_PHAQI_weekpart_Country_Lodge_Pasig_EMBNCR.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline\daily_PM25_and_PHAQI_weekpart_Dela_Paz_Barangay_Hall_Pasig_City.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline\daily_PM25_and_PHAQI_weekpart_ICE_Pasig.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline\daily_PM25_and_PHAQI_weekpart_Manggahan_Barangay_Hall_Pasig_City.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\temporal_heatmaps_guideline\daily_PM25_and_PHAQI_weekpart_Maybunga_Barangay_Hall_Pasig_City.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Ou